# Numerical integration

Many integrals are analytically intractable, and must be performed numerically.  Here, we show how Python can be used to numerically integrate symbolic functions and non-symbolic functions, which correspond to an array of values defined with respect to a grid.

## Symbolic functions

### One dimension

Say you want to perform some definite integral that can be expressed in terms of known functions.  For example, let's consider the integral
\begin{align}
\int_0^\infty \left( \frac{\tanh x}{x} \right)^2 dx
\end{align}
A common and powerful method to perform numerical integration is quadrature, which is part of Python's `scipy` library.  Let's import `numpy`, `scipy`'s integration library (we'll import it as `spi`), and define the function `f` to integrate.

In [1]:
import numpy as np
import scipy.integrate as spi

def f(x):
    f = (np.tanh(x)/x)**2
    return f

The syntax for the `quad` function is simple; input the function, the lower limit, and the upper limit. Note that $\infty$ exists in the `numpy` library as `numpy.inf`.

In [2]:
spi.quad(f,0,np.inf,epsrel=1e-8)

(1.705113595270016, 3.320359385696755e-09)

The `quad` function outputs what's known as a "tuple" in Python.  It's like an array, but you can't manipulate it.  The first element is the value of the integral, and the second element is the relative error.  The `epsrel` parameter controls the error.  If you make `epsrel` larger, you will see the relative error go up.

## Two dimensions

Now, say you wanted to perform a 2D integral, such as
\begin{align}
\int_0^{2\pi} d \phi \int_0^\infty \rho d\rho \cos^2(2\phi) \frac{1}{\rho^2} J_2(\rho) e^{-\frac{\rho^2}{2}}
\end{align}
where $J_2$ is the 2$^\mathrm{nd}$ order Bessel function of the 1$^\mathrm{st}$ kind; we can call this with the `scipy` function `jn`, which exists in the library `scipy.special`.

In [3]:
import scipy.special as sps

def f(rho,phi):
    f = np.cos(2*phi)**2*sps.jn(2,rho)*np.exp(-rho**2/2)/rho
    return f

spi.dblquad(f,0,2*np.pi,lambda phi: 0,lambda phi: np.inf, epsrel=1e-8)

(0.3346759379352825, 1.1774256975833028e-08)

The inputs to `dblquad` are a bit funny.  They are, in order, the integrand function, the lower limit of the function's 2$^\mathrm{nd}$ argument, the upper limit of the function's 2$^\mathrm{nd}$ argument, the lower limit of the function's 1$^\mathrm{st}$ argument, and the upper limit of the function's 1$^\mathrm{st}$ argument.  It is set up so that the limits of integration of the function's 1$^\mathrm{st}$ argument can depend on the function's 2$^\mathrm{nd}$ argument!  Also, the `lambda` syntax provides a different way of defining a function in Python.  So, writing `lambda rho: 0` means that the lower limit of the $\phi$ integral is a function of $\rho$, and is equal to $0$.

For the case we considered, of course, these limits are not actually functions of $\rho$.  Let's consider an integral where this functionality will be helpful.
\begin{align}
\int_{-3}^3 dx \int_{-\sqrt{9-x^2}}^{\sqrt{9-x^2}} dy.
\end{align}
Of course, we CAN do this integral analytically, but we'll do it numerically for the sake of example.  Remember, to use `dblquad`, the integration limits of the 1$^\mathrm{st}$ variable should depend on the 2$^\mathrm{nd}$ variable.  So we'll define our integrand function to be $f(y,x) = 1$.

In [4]:
def f(x,y):
    return 1.0

spi.dblquad(f,-3,3, lambda x: -np.sqrt(9-x**2) ,lambda x: np.sqrt(9-x**2), epsrel=1e-8)

(28.274333882308188, 1.8004239876745487e-08)

Notice that this integral corresponds to the area of a sphere of radius 3!  Of course, $\pi 3^2 \simeq 28.27$.